In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import random
import time

In [2]:
flow = pd.read_excel('Flow.xlsx')
cost = pd.read_excel('Costs.xlsx')

cost_criterion = cost.mul(flow)

capacity = pd.read_excel('Capacity.xlsx', header = None)
capacity.set_index(0, inplace = True)
capacity.columns = ['Flow']
capacity_threshold = capacity.loc['Large Capacity','Flow']

fixed_cost = pd.read_excel('FixedCosts.xlsx')
fixed_cost.set_index('Size', inplace = True)

In [3]:
'''Initialisation, Feasibility and Cost functions'''
def initialisation(nodes):
    # Randomly select a predefined number of nodes to act as hubs
    hubs = np.random.choice(nodes, size = number_of_hubs, replace = False)
    # Keep only spokes (non-hub nodes)
    nonhubs = np.array(list(set(nodes) - set(hubs)))
    hub_allocation = nodes.copy()
    # Assign each spoke randomly to a hub
    for i in nonhubs:
        hub_allocation[i] = np.random.choice(hubs, size = 1)
    return hub_allocation

def check_hub_capacity_flow(capacity_threshold, hub_allocation, flow):
    hubs = list(set(hub_allocation))
    # Check flow for each hub
    for hub in hubs:
        sum_flow = 0
        # For calculating the flow take into account only the hub and its connected spokes
        hub_nodes = list(np.where(hub_allocation == hub)[0])
        for hnode in hub_nodes:
            sum_flow += flow.iloc[hnode,:].sum()
        # If flow exceeds capacity label solution as infeasible
        if sum_flow > capacity_threshold:
            return 'Infeasible'
    return 'Feasible'

def cost_function(flow, cost, sol, a, nodes):
    if type(sol) != np.ndarray:
        sol = np.array(sol)
    total_cost = 0
    for i in nodes:
        for j in nodes:
            total_cost += flow.iloc[i,j] * (cost.iloc[i,sol[i]] +  a * cost.iloc[sol[i],sol[j]] + 
                                            cost.iloc[sol[j],j])

    hubs = list(set(sol))
    # For each hub check flow to assign equivalent size cost
    for hub in hubs:
        sum_flow = 0
        hub_nodes = list(np.where(sol == hub)[0])
        for hnode in hub_nodes:
            sum_flow += flow.iloc[hnode,:].sum()
        if sum_flow <= capacity.loc['Small Capacity','Flow']:
            total_cost += fixed_cost.loc['Small','Cost']
        elif sum_flow <= capacity.loc['Medium Capacity','Flow']:
            total_cost += fixed_cost.loc['Medium','Cost']
        else:
            total_cost += fixed_cost.loc['Large','Cost']
    return total_cost

'''Parents Selection - Unbiased Tournament Selection Function'''
def selection(population, number):
    df_tournament = population.copy()
    df_tournament.sort_values(by='cost', inplace = True)
    df_tournament.reset_index(drop = True, inplace = True)
    df_tournament.columns = ['cont1', 'cost1']
    
    # Keep best solution
    elite = df_tournament.loc[0, 'cont1']
    
    # Define number of permutations
    perm_num = number - 1
    
    # Create tournament contestants and their fit, the tournament takes place
    # between contestants in the same row
    for i in range(perm_num):
        df_tournament['cont'+str(i+2)] = np.random.permutation(df_tournament['cont1'].values)
        df_tournament['cost'+str(i+2)] = df_tournament['cont'+str(i+2)].map(lambda x: cost_function(flow, cost, x, alpha, nodes))

    def winner(x):
        # Create a DataFrame with the contestants and their fit to find a winner
        contestants = []
        costs = []
        for idx in x.index:
            if 'cont' in idx:
                contestants.append(idx)
            else:
                costs.append(idx)

        df_slot = pd.DataFrame({'contestants': x[contestants].values, 'costs':x[costs].values}, 
                               index = list(range(1,number+1)))

        min_cost = df_slot['costs'].min()
        winner = df_slot[df_slot['costs'] == min_cost]['contestants'].values[0]

        return winner

    df_tournament['winners'] = df_tournament.apply(winner, axis = 1)
    winners = df_tournament['winners'].values.tolist()
    random.shuffle(winners)

    return winners, elite

'''Crossover Function'''
# Single Point Crossover Operation

def crossover(mother, father, nodes, number_of_hubs):
    # Probability of crossover happening
    execute = np.random.choice([0,1], size = 1, p = [0.05, 0.95])
    mother = np.array(mother)
    father = np.array(father)

    if execute == 0:
        return mother
    else:
        # Select a random index to split the parents and connect them
        split = np.random.choice(nodes)
        child = mother.copy()
        child[split:] = father[split:]
        child_hubs = set(child)
        # Costraints to ensure feasible offsprings
        if len(child_hubs) > number_of_hubs:
            child_hubs = []
            for i in range(len(child)):
                if child[i] == i:
                    child_hubs.append(i)
            if len(child_hubs) > number_of_hubs:
                child_hubs = np.random.choice(list(set(child_hubs)), number_of_hubs, replace = False)
            if len(child_hubs) == 0:
                return mother
            broken_nodes = []
            for i in range(len(child)):
                if child[i] not in child_hubs:
                    broken_nodes.append(i)
            for broken in broken_nodes:
                child[broken] = np.random.choice(child_hubs)
        while len(set(child)) < number_of_hubs:
            new_possible_hubs = list(set(nodes) - set(child))
            new_child_hub = np.random.choice(new_possible_hubs)
            child[new_child_hub] = new_child_hub
        for hub in child_hubs:
            if child[hub] != hub:
                child[hub] = hub
        return child

'''Mutation Function'''
def mutation(child, nodes):
    # Probability of mutation happening
    execute = np.random.choice([0,1], size = 1, p = [0.95, 0.05])
        
    if execute == 0:
        return child
    else:
        # Randomly select a spoke and assign it to the hub with the lowest 
        # transportation cost
        mutated_child = np.array(child).copy()
        child_nodes = np.array(list(set(nodes) - set(mutated_child)))
        hubs_size = Counter(mutated_child)

        mutated_node = np.random.choice(child_nodes)
        mutation_hub = mutated_child[mutated_node]
        while hubs_size[mutation_hub] <= 1:
            new_child_nodes = child_nodes[child_nodes != mutated_node]
            mutated_node = np.random.choice(new_child_nodes)
            mutation_hub = child[mutated_node]

        alt_hubs = list(set(mutated_child))
        alt_hubs.remove(mutation_hub)
        hub_cost = []
        for hub in alt_hubs:
            hub_cost.append(cost_criterion.iloc[mutated_node, hub])

        min_cost = hub_cost.index(min(hub_cost))
        new_hub = alt_hubs[min_cost]
        mutated_child[mutated_node] = new_hub
        return mutated_child

'''Repair Mechanism Function'''
def repair_chromosome(chromosome):
    hubs = list(set(chromosome))
    hub_flow = []

    for hub in hubs:
        sum_flow = 0
        hub_nodes = list(np.where(chromosome == hub)[0])
        for hnode in hub_nodes:
            sum_flow += flow.iloc[hnode,:].sum()
        hub_flow.append(sum_flow)

    check = list(np.array(hub_flow) > capacity_threshold)
    if True in check:
        broken_chromosome = chromosome.copy()

        df_flow = pd.DataFrame({'hubs':hubs, 'flow':hub_flow})
        min_flow_hub = df_flow[df_flow['flow'] == df_flow['flow'].min()]['hubs'].values[0]

        overflow_hub = hubs[check.index(True)]
        overflow_hub_nodes = list(np.where(broken_chromosome == overflow_hub)[0])
        overflow_hub_nodes.remove(overflow_hub)
        node_flow = []
        for hnode in overflow_hub_nodes:
            node_flow.append(flow.iloc[hnode,:].sum())
        max_flow_idx = node_flow.index(max(node_flow))
        max_node_flow = overflow_hub_nodes[max_flow_idx]

        fixed_chromosome = broken_chromosome.copy()
        fixed_chromosome[max_node_flow] = min_flow_hub

        valid_chromosome = check_flow(capacity_threshold, fixed_chromosome, flow, list(set(fixed_chromosome)), nodes)
        if valid_chromosome == 'Feasible':
            return fixed_chromosome
        else:
            return np.nan
    else:
        return chromosome

In [4]:
'''Input Variables'''
nodes = np.array(range(flow.shape[0]))
print('Number of hubs:')
number_of_hubs = int(input())
print('Alpha:')
alpha = float(input())
print('Population size:')
population_size = int(input())
print('Generations:')
generations = int(input())

Number of hubs:
3
Alpha:
0.2
Population size:
80
Generations:
60


In [9]:
start = time.time()

'''Population Initialization'''
population = []
for i in range(population_size):
    hub_allocation = initialisation(nodes)
    feasibility = check_hub_capacity_flow(capacity_threshold, hub_allocation, flow)

    while feasibility == 'Infeasible': 
        hub_allocation = initialisation(nodes)
        feasibility = check_hub_capacity_flow(capacity_threshold, hub_allocation, flow)

    while hub_allocation.tolist() in population:
        hub_allocation = initialisation(nodes)
        feasibility = check_hub_capacity_flow(capacity_threshold, hub_allocation, flow)

        while feasibility == 'Infeasible': 
            hub_allocation = initialisation(nodes)
            feasibility = check_hub_capacity_flow(capacity_threshold, hub_allocation, flow)

    population.append(hub_allocation.tolist())

df_initial_population = pd.DataFrame({'chromosomes':population})
df_initial_population['cost'] = df_initial_population['chromosomes'].map(lambda x: cost_function(flow, cost, np.array(x), alpha, nodes))
initial_cost_sorted = df_initial_population.sort_values(by='cost').copy()
initial_cost_sorted.reset_index(drop = True, inplace = True)

breakout = 0
for gen in range(generations):
    breakout += 1
    df_population = pd.DataFrame({'chromosomes':population})

    '''Fitness Function Calculation'''
    df_population['cost'] = df_population['chromosomes'].map(lambda x: cost_function(flow, cost, x, alpha, nodes))

    '''Tournament Selection'''
    intermid_population, elite_chromosome = selection(df_population, 3)
    cost_sorted = df_population.sort_values(by='cost').copy()
    cost_sorted.reset_index(drop = True, inplace = True)

    '''Mating'''
    '''Crossover'''
    crossover_population = []
    for i in range(len(intermid_population)-1):
        crossover_population.append(crossover(intermid_population[i], intermid_population[i+1], 
                                              nodes, number_of_hubs))

    '''Mutation'''
    df_generation = pd.DataFrame({'crossover_population':crossover_population})
    df_generation['mutated_population'] = df_generation['crossover_population'].map(lambda x: mutation(x, nodes))
    df_generation.drop('crossover_population', axis = 1, inplace = True)

    '''Repair'''
    df_generation['new_gen'] = df_generation['mutated_population'].map(repair_chromosome)
    if df_generation['new_gen'].isnull().sum() > 0:
        faulty_sols_len = df_generation[df_generation['new_gen'].isnull()].shape[0]
        for i,j in zip(range(1,faulty_sols_len+1),df_generation[df_generation['new_gen'].isnull()].index):
            df_generation.loc[j, 'new_gen'] = cost_sorted.loc[i, 'chromosomes']

    df_generation.drop('mutated_population',axis = 1, inplace = True)

    '''Elitism'''
    new_population =  list(df_generation['new_gen'].values)
    new_population.append(np.array(elite_chromosome))
    df_generation = pd.DataFrame({'chromosomes':new_population})
    population = df_generation['chromosomes'].values.tolist()

    '''Break'''
    previous_population = cost_sorted.copy()

    new_population = df_generation.copy()
    new_population['cost'] = new_population['chromosomes'].map(lambda x: cost_function(flow, cost, x, alpha, nodes))
    new_population.sort_values(by='cost', inplace = True)
    new_population.reset_index(drop = True, inplace = True)
    if new_population.loc[0, 'cost'] < previous_population.loc[0, 'cost']:
        breakout = 0
    if breakout >= 15:
        last_gen = gen
        break

end = time.time()    
final_df = df_generation.copy()
final_df['cost'] = final_df['chromosomes'].map(lambda x: cost_function(flow, cost, x, alpha, nodes))
final_df.sort_values(by='cost', inplace = True)
final_df.reset_index(drop = True, inplace = True)
optimal_solution = final_df.loc[0,'chromosomes']
optimal_cost = final_df.loc[0,'cost']
run_time = round(end-start, 2)

print('***Results***')
print('Time required to find optimal solution: ', run_time, ' seconds.')
print('Number of generations required to reach optimal solution:', last_gen - breakout)
print('The optimal solution / allocation of hubs and spokes is: ', list(optimal_solution))
print('The cost of the optimal solution is: ', optimal_cost)

***Results***
Time required to find optimal solution:  52.4  seconds.
Number of generations required to reach optimal solution: 11
The optimal solution / allocation of hubs and spokes is:  [5, 5, 5, 3, 3, 5, 6, 6, 5, 6]
The cost of the optimal solution is:  645192795.5108399
